# $P(z|{\rm DM})$

In [1]:
# imports
from scipy.stats import norm, lognorm

import numpy as np
from importlib import reload

from frb.figures import utils as ffutils
from frb.dm import prob_dmz
from frb import frb

from bokeh import plotting
from bokeh import models

/Users/ckilpatrick/scripts/anaconda3/envs/astroconda37/lib/python3.7/site-packages/FRB-0.1.dev0-py3.7.egg/frb/galaxies/nebular.py:18: UserWarning: extinction package not loaded.  Extinction corrections will fail
  warnings.warn("extinction package not loaded.  Extinction corrections will fail")


# $\rm  DM_{ISM}$

## Normal

In [2]:
rv = norm(loc=30., scale=3.)

In [3]:
x = np.linspace(0., 100., 1000)

In [4]:
# Plot
plotting.output_notebook()

p = plotting.figure(title='PDF for DM_ISM', x_axis_label='DM_ISM',
                    #x_axis_type="log", y_axis_type="log",
                   y_axis_label='P(DM_ISM)')#, x_range=(10**-4, 1.),
                   #y_range=(1e-12, 1))

# Plot em
p.line(x, rv.pdf(x), legend='PDF', line_width=2)

p.legend.location = "top_right"

ffutils.set_bokeh_fontsize(p, 16)

plotting.show(p)

Loading BokehJS ...

# $\rm DM_{host}$

## log-normal

In [5]:
p_host = lognorm(s=0.88, loc=0., scale=68.2)
x_host = np.linspace(0., 1000., 1000)

In [6]:
# Plot
plotting.output_notebook()

p = plotting.figure(title='PDF for DM_host', x_axis_label='DM_host',
                    #x_axis_type="log", y_axis_type="log",
                   y_axis_label='P(DM_host)')#, x_range=(10**-4, 1.),
                   #y_range=(1e-12, 1))

# Plot em
p.line(x_host, p_host.pdf(x_host), legend='PDF Host', line_width=2)

p.legend.location = "top_right"

ffutils.set_bokeh_fontsize(p, 16)

plotting.show(p)

Loading BokehJS ...

In [7]:
p_host.pdf(-10.)

0.0

# $P({\rm DM_{\rm cosmic}}|FRB)$

## FRB 180924

In [15]:
from astropy import units as u
frb180924 = frb.FRB.by_name('FRB180924')
print(frb180924)
print(dir(frb180924))
print(frb180924.DM)
frb180924.DM=273.5*u.pc/u.cm**3
print(frb180924)
frb180924.frb_name='FRB20190617B'
print(frb180924)

<FRB: FRB180924 J214425.255-405400.1 DM=362.16 pc / cm3 z=0.3212>
['DM', 'DMISM', 'DMISM_err', 'DM_err', 'FRB', 'RM', 'RM_err', 'S', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'by_name', 'coord', 'cosmo', 'eellipse', 'fluence', 'fluence_err', 'frb_name', 'from_dict', 'from_json', 'grab_host', 'lpol', 'lpol_err', 'main_dict', 'make_outfile', 'nu_c', 'pulse', 'refs', 'repeater', 'set_DMISM', 'set_ee', 'set_pulse', 'sig_a', 'sig_b', 'write_to_json', 'z']
362.16 pc / cm3
<FRB: FRB180924 J214425.255-405400.1 DM=273.5 pc / cm3 z=0.3212>
<FRB: FRB20190617B J214425.255-405400.1 DM=273.5 pc / cm3 z=0.3212>


In [16]:
reload(prob_dmz)
DM_cosmic, P_DM_cosmic = prob_dmz.prob_DMcosmic_FRB(frb180924)

In [17]:
# Plot
plotting.output_notebook()

p = plotting.figure(title='PDF for DM_cosmic', x_axis_label='DM_cosmic',
                    #x_axis_type="log", y_axis_type="log",
                   y_axis_label='P(DM_cosmic)', x_range=(0., 500))
                   #y_range=(1e-12, 1))

# Plot em
p.line(DM_cosmic, P_DM_cosmic, legend='FRB 180924', line_width=2)

p.legend.location = "top_right"

ffutils.set_bokeh_fontsize(p, 16)

plotting.show(p)

Loading BokehJS ...

# $P({\rm DM_{\rm cosmic}|z})$ grid

## Build

In [25]:
reload(prob_dmz)
z, DMcosmic, P_DMcosmic_z = prob_dmz.grid_P_DMcosmic_z()

KeyboardInterrupt: 

In [20]:
P_DMcosmic_z.shape

(1000, 200)

In [21]:
np.max(P_DMcosmic_z)

1.0

In [27]:
logP = np.log10(P_DMcosmic_z)
logP[np.invert(np.isfinite(logP))] = -8.

from scipy.interpolate import interp1d
prob = interp1d(DM_cosmic, P_DM_cosmic, bounds_error=False, fill_value='extrapolate')

/Users/ckilpatrick/scripts/anaconda3/envs/astroconda37/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log10
  """Entry point for launching an IPython kernel.


## Show

In [23]:
min_x, max_x = 0., 2.
min_y, max_y = 1, 5000.
p1 = plotting.figure(title="log10 PDF DM_cosmic vs. z", plot_width=int(700), plot_height=int(600),
            x_range = [min_x, max_x], y_range = [min_y, max_y],
                    x_axis_label='z', y_axis_label='DM_cosmic',)

# Fill in the missing parameters to use the `image` renderer to
# display the Mandelbrot image color mapped with the palette 'Spectral11'
#
# NOTE: the `image` renderer can display many images at once, so it takes
# **lists** of images, coordinates, and palettes. Remember to supply sequences
# for these parameters, even if you are just supplying one.

color_mapper = models.LinearColorMapper(palette="Viridis256", low=-5, high=0.)

p1.image(image=[logP],             # image data
         x=[min_x],               # lower left x coord
         y=[min_y],               # lower left y coord
         dw=[max_x-min_x],        # *data space* width of image
         dh=[max_y-min_y],        # *data space* height of image
         color_mapper = color_mapper,
         #palette="Spectral11",    # palette name
)

color_bar = models.ColorBar(color_mapper=color_mapper, #ticker=models.LogTicker(),
                     label_standoff=5, border_line_color=None, location=(0,0),
                           title='log10 PDF',  major_label_text_font_size='12px')

p1.add_layout(color_bar, 'right')

ffutils.set_bokeh_fontsize(p1, 16)

plotting.show(p1)

In [28]:
print(logP)
print(logP.shape)

[[ 0.         -8.         -8.         ... -8.         -8.
  -8.        ]
 [-8.         -0.12555474 -1.72147903 ... -8.         -8.
  -8.        ]
 [-8.         -0.83667701 -0.27120786 ... -8.         -8.
  -8.        ]
 ...
 [-8.         -8.79335127 -8.16291384 ... -4.63221682 -4.62844347
  -4.62467498]
 [-8.         -8.79465716 -8.16421973 ... -4.63374071 -4.62997138
  -4.62620696]
 [-8.         -8.79596173 -8.16552431 ... -4.63526244 -4.63149711
  -4.62773674]]
(1000, 200)


In [39]:
DM_val=1.0+(5000-1.0)/1000*np.arange(1000)
z_val=0.0+(2.0)/200*np.arange(200)
print(DM_val.shape)
prob_z=None
for i,val in enumerate(DM_val):
    probability=prob(val)
    if prob_z is None:
        prob_z = probability*10**logP[i,:]
    else:
        prob_z += probability*10**logP[i,:]

print(prob_z.shape)

(1000,)
(200,)


In [48]:
# Plot
plotting.output_notebook()

print(np.sum(prob_z))
prob_z = prob_z / np.sum(prob_z)

tot=0
for i in np.arange(len(prob_z)):
    tot+=prob_z[i]
    if tot>0.997:
        print(i,z_val[i])
        break

p = plotting.figure(title='PDF for z|DM', x_axis_label='z',
                    #x_axis_type="log", y_axis_type="log",
                   y_axis_label='P(z|DM)', x_range=(0., 1.0))
                   #y_range=(1e-12, 1))

# Plot em
p.line(z_val, prob_z, legend='FRB20190617B', line_width=2)

p.legend.location = "top_right"

ffutils.set_bokeh_fontsize(p, 16)

plotting.show(p)

Loading BokehJS ...

1.0
36 0.36
